---
## Data prep steps : preprocess dataset into a list of  dspy.Example


In [1]:
import pandas as pd
#xls = pd.ExcelFile('/workspace/nvdata/Eval data set.xlsx')
#df1 = pd.read_excel(xls, 'Single Bug Details')
train=pd.read_csv('/workspace/nvdata/train.csv')
n=len(train)
num2answer=[]
for i in range(n):
    label=int(train.iloc[i,-1])+2
    answer=train.iloc[i,label]
    num2answer.append(answer)
train["answer"]=num2answer
train.head()

,Unnamed: 0,startphrase,ending0,ending1,ending2,ending3,label,answer
0,0,Which factor will most likely cause a person t...,a leg muscle relaxing after exercise,a bacterial population in the bloodstream,several viral particles on the skin,carbohydrates being digested in the stomach,1,a bacterial population in the bloodstream
1,1,Lichens are symbiotic organisms made of green ...,carbon dioxide,food,protection,water,1,food
2,2,When a switch is used in an electrical circuit...,cause the charge to build.,increase and decrease the voltage.,cause the current to change direction.,stop and start the flow of current.,3,stop and start the flow of current.
3,3,Which of the following is an example of an ass...,contact lens,motorcycle,raincoat,coffee pot,0,contact lens
4,4,"Rocks are classified as igneous, metamorphic, ...",their color,their shape,how they formed,the minerals they contain,2,how they formed


In [2]:
#df=train.iloc[:,[1,2,3,4,5,7]]
def row_bind(row):
    return [row['ending0'],row['ending1'],row['ending2'],row['ending3']]
train['choices']=train.apply(row_bind, axis=1)
train.head()


,Unnamed: 0,startphrase,ending0,ending1,ending2,ending3,label,answer,choices
0,0,Which factor will most likely cause a person t...,a leg muscle relaxing after exercise,a bacterial population in the bloodstream,several viral particles on the skin,carbohydrates being digested in the stomach,1,a bacterial population in the bloodstream,"[ a leg muscle relaxing after exercise , a ba..."
1,1,Lichens are symbiotic organisms made of green ...,carbon dioxide,food,protection,water,1,food,"[ carbon dioxide , food , protection , water]"
2,2,When a switch is used in an electrical circuit...,cause the charge to build.,increase and decrease the voltage.,cause the current to change direction.,stop and start the flow of current.,3,stop and start the flow of current.,"[ cause the charge to build. , increase and d..."
3,3,Which of the following is an example of an ass...,contact lens,motorcycle,raincoat,coffee pot,0,contact lens,"[ contact lens , motorcycle , raincoat , co..."
4,4,"Rocks are classified as igneous, metamorphic, ...",their color,their shape,how they formed,the minerals they contain,2,how they formed,"[ their color , their shape , how they forme..."


In [3]:
trainset=train.iloc[:50,:]
trainset.head()

,Unnamed: 0,startphrase,ending0,ending1,ending2,ending3,label,answer,choices
0,0,Which factor will most likely cause a person t...,a leg muscle relaxing after exercise,a bacterial population in the bloodstream,several viral particles on the skin,carbohydrates being digested in the stomach,1,a bacterial population in the bloodstream,"[ a leg muscle relaxing after exercise , a ba..."
1,1,Lichens are symbiotic organisms made of green ...,carbon dioxide,food,protection,water,1,food,"[ carbon dioxide , food , protection , water]"
2,2,When a switch is used in an electrical circuit...,cause the charge to build.,increase and decrease the voltage.,cause the current to change direction.,stop and start the flow of current.,3,stop and start the flow of current.,"[ cause the charge to build. , increase and d..."
3,3,Which of the following is an example of an ass...,contact lens,motorcycle,raincoat,coffee pot,0,contact lens,"[ contact lens , motorcycle , raincoat , co..."
4,4,"Rocks are classified as igneous, metamorphic, ...",their color,their shape,how they formed,the minerals they contain,2,how they formed,"[ their color , their shape , how they forme..."


In [4]:
df=trainset.iloc[:,[1,-1,-2]]
df.rename(columns={'startphrase': 'question'}, inplace=True)

trainset=[dict(v) for _, v in df.iterrows()]

trainset[:2]


/tmp/ipykernel_85116/2985071205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'startphrase': 'question'}, inplace=True)


[{'question': 'Which factor will most likely cause a person to develop a fever? ',
  'choices': [' a leg muscle relaxing after exercise ',
   ' a bacterial population in the bloodstream ',
   ' several viral particles on the skin ',
   ' carbohydrates being digested in the stomach'],
  'answer': ' a bacterial population in the bloodstream '},
 {'question': 'Lichens are symbiotic organisms made of green algae and fungi. What do the green algae supply to the fungi in this symbiotic relationship? ',
  'choices': [' carbon dioxide ', ' food ', ' protection ', ' water'],
  'answer': ' food '}]

In [5]:
## arrange data to be a List of Dspy Examples
import random
import dspy


kwargs = dict(fields=("question", "choices", "answer"), input_keys=("question",))
hover = [
    dspy.Example(question=x['question'], choices=x['choices'], answer=x["answer"]).with_inputs("question")
    for x in trainset
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:30], hover[30:40], hover[40:]

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
testset[:3]

[Example({'question': 'Which of the following properties provides the BEST way to identify a mineral? ', 'choices': [' hardness ', ' shape ', ' size ', ' temperature'], 'answer': ' hardness '}) (input_keys={'question'}),
 Example({'question': 'Which type of energy in gasoline is transformed into mechanical energy in a motorcycle engine? ', 'choices': [' chemical ', ' nuclear ', ' magnetic ', ' electrical'], 'answer': ' chemical '}) (input_keys={'question'}),
 Example({'question': 'Peach trees have sweet-smelling blossoms and produce rich fruit. What is the main purpose of the flowers of a peach tree? ', 'choices': [' to attract bees for pollination ', ' to create flower arrangements ', ' to protect the tree from disease ', ' to feed migratory birds'], 'answer': ' to attract bees for pollination '}) (input_keys={'question'})]

---
## naive run = no optimization just directly predict 

In [7]:

import dspy
import os
from dotenv import load_dotenv
load_dotenv()
base_url = 'https://integrate.api.nvidia.com/v1'
model = "meta/llama-3.3-70b-instruct"

llama33_70b = dspy.LM(f'openai/{model}', api_key=os.environ["NVIDIA_API_KEY"], api_base=base_url)

mixtral8x22b = dspy.LM('openai/nvdev/mistralai/mixtral-8x22b-instruct-v0.1', api_key=os.environ["NV_DEV_API_KEY"], api_base=base_url, temperature=0.7)

dspy.configure(lm=mixtral8x22b)

In [8]:
example=trainset[0]
print("the example is " , example.question , '\n', "choices are : ", example.choices )
module = dspy.ChainOfThought("question, choices -> answer")
out=module(question=example.question, choices=example.choices)
print(" the predicted output is >>> ", out, "\nthe correct answer is : ", example.answer)

the example is  When plants undergo photosynthesis, a reaction produces sugar, oxygen, and water. During respiration, stored energy from the products of photosynthesis is converted to usable energy. In what form is the energy stored prior to use in respiration?  
 choices are :  [' chemical energy ', ' kinetic energy ', ' light energy ', ' heat energy']
 the predicted output is >>>  Prediction(
    reasoning='During photosynthesis, plants convert light energy into chemical energy in the form of glucose (sugar). This chemical energy is stored in the glucose molecules, which are then broken down during respiration to release the stored energy to be used by the plant for growth and other functions.',
    answer='[1] « chemical energy »'
) 
the correct answer is :   chemical energy 


In [9]:
eg=" like they are pumped up» "
eg=eg.strip()
eg


'like they are pumped up»'

## create our own custom metrics for measuring 

In [32]:
from colorama import Fore
def validate_answer(example, pred, trace=None, return_answer=False):
    print(Fore.BLUE+ "inspect reasoning >>> " , pred.reasoning , return_answer)
    
    ## stripping spaces before or after the answer 
    answer=example.answer.strip()
    pred_answer=pred.answer 
    if '[' in pred_answer or ']' in pred_answer:
        pred_answer=pred_answer.split(']')[-1]
        pred_answer=pred_answer.replace("«","").replace("»","").strip() 
        print(Fore.CYAN + "stripped strange characters in pred if any :", pred_answer )
        print(Fore.BLACK + "gold answer:",  example.answer )
        print( answer.lower()==pred_answer.lower() )
    else:
        pred_answer=pred_answer.replace("« ","").replace(" »","").strip()
        print(Fore.CYAN + "stripped strange characters in pred if any :", pred_answer )
        print(Fore.BLACK + "gold answer:" ,example.answer )
        print( answer.lower()==pred_answer.lower() )
    if return_answer :
        return answer.lower() == pred_answer.lower(), pred_answer 
    else:
        return answer.lower() == pred_answer.lower()

In [33]:
scores = []
i=0
for x in testset:
    if i==0 :
        pred = module(question=x.question, choices=x.choices)
        true_false_flag, pred_answer  = validate_answer(x, pred,None,return_answer=True)
        print("---"*10)
        print(pred_answer)
        print(x.answer)
        print(true_false_flag)
    else:
        break 
    if true_false_flag:
        scores.append(1)
    else:
        scores.append(0)
    i+=1 
scores

inspect reasoning >>>  The best way to identify a mineral is by its physical and chemical properties. Hardness is a physical property that can be used to distinguish minerals. The Mohs scale of mineral hardness is a qualitative ordinal scale that characterizes the scratch resistance of various minerals through the ability of a harder material to scratch a softer material. This scale is useful for field geology as a way to identify minerals using scratch kits. True
stripped strange characters in pred if any : hardness
gold answer:  hardness 
True
------------------------------
hardness
 hardness 
True


[1]

In [44]:
q=devset[0].question


In [45]:
[example.choices for example in devset if example.question==q][0]

[' removing grass from the steepest slope. ',
 ' building terraces into the sides of a slope. ',
 ' heavily watering the vegetation on the slope. ',
 ' increasing the slope of the land by adding more soil.']

In [50]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    choices = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")
class MultiChoices(dspy.Module):
    def __init__(self, dataset):
        #self.pred = dspy.ChainOfThought('question, choices -> answer')
        self.pred = dspy.ChainOfThought(BasicQA)
        self.dataset=dataset

    def forward(self, question):
        choices=[example.choices for example in self.dataset if example.question==question][0]
        return self.pred(choices=choices, question=question)

In [46]:
example

Example({'question': 'When plants undergo photosynthesis, a reaction produces sugar, oxygen, and water. During respiration, stored energy from the products of photosynthesis is converted to usable energy. In what form is the energy stored prior to use in respiration? ', 'choices': [' chemical energy ', ' kinetic energy ', ' light energy ', ' heat energy'], 'answer': ' chemical energy '}) (input_keys={'question'})

In [51]:
qa_bot = MultiChoices([example])
pred = qa_bot.forward(example.question)
pred.answer

'chemical energy'

In [53]:
pred.reasoning

'During photosynthesis, plants convert light energy into chemical energy. This chemical energy is stored in the form of glucose, which is produced from carbon dioxide and water. During respiration, this stored chemical energy is released and converted to usable energy in the form of ATP (adenosine triphosphate).'

In [76]:
from dspy.evaluate import Evaluate

THREADS = 5
kwargs = dict(num_threads=THREADS, display_progress=True, display_table=10)
evaluate = dspy.Evaluate(devset=trainset, metric=validate_answer, num_threads=16, display_progress=True, display_table=5)

evaluate(MultiChoices(trainset))

inspect reasoning >>>  During photosynthesis, plants convert light energy into chemical energy. This chemical energy is stored in the form of glucose, which is produced from carbon dioxide and water. During respiration, this stored chemical energy is released and converted to usable energy in the form of ATP (adenosine triphosphate). False
stripped strange characters in pred if any : chemical energy
gold answer:  chemical energy 
True
Average Metric: 1.00 / 1 (100.0%):   0%|                                                                                                                                                  | 0/30 [00:00<?, ?it/s]inspect reasoning >>>  All cells, regardless of type, share the characteristic of requiring energy to perform various cellular functions. This energy is obtained through different means such as photosynthesis, cellular respiration, or consuming other organisms. False
stripped strange characters in pred if any : They need energy.
gold answer:  They ne

2025/01/09 12:56:22 INFO dspy.evaluate.evaluate: Average Metric: 26 / 30 (86.7%)


,question,choices,example_answer,reasoning,pred_answer,validate_answer
0,"When plants undergo photosynthesis, a reaction produces sugar, oxy...","[ chemical energy , kinetic energy , light energy , heat energy]",chemical energy,"During photosynthesis, plants convert light energy into chemical e...",chemical energy,✔️ [True]
1,Lichens are symbiotic organisms made of green algae and fungi. Wha...,"[ carbon dioxide , food , protection , water]",food,"In a lichen, green algae and fungi work together to create a symbi...",food,✔️ [True]
2,Which best describes why winds at the beach change direction betwe...,"[' because the temperatures over land and over water change ', ' b...",because the temperatures over land and over water change,The change in wind direction at the beach between day and night is...,« because the temperatures over land and over water change »,✔️ [True]
3,"Many natural rock formations change color over time. In Utah, for ...","[ chemical weathering , mechanical weathering , water erosion , ...",chemical weathering,"The process of iron oxidizing and forming red, orange, and yellow ...",chemical weathering,✔️ [True]
4,A cow and a frog have similar bone structures in their forelimbs. ...,"[' Both have undergone similar mutations. ', ' Both descended from...",Both descended from a common ancestor.,"The similarity in bone structure between a cow and a frog, despite...",Both descended from a common ancestor.,✔️ [True]


86.67

In [36]:
dspy.inspect_history()





[2025-01-09T12:10:30.970394]

System message:

Your input fields are:
1. `question` (str)
2. `choices` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## choices ## ]]
{choices}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, `choices`, produce the fields `answer`.


User message:

[[ ## question ## ]]
Scientists use the term "light year" to describe 

[[ ## choices ## ]]
[1] « how fast light travels in space. »
[2] « the distance light travels in one year. »
[3] « how large the diameters of different stars are. »
[4] « the time it takes the Sun to move through the galaxy.»

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and the

---
## optimize using KNNFewShot

In [58]:
## manually update the dsp package to aboid this error : AttributeError: module 'dspy.dsp' has no attribute 'SentenceTransformersVectorizer'
!pip uninstall dsp-ml

!pip install git+https://github.com/stanfordnlp/dsp.git 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/stanfordnlp/dsp.git to /tmp/pip-req-build-709q4or9
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/dsp.git /tmp/pip-req-build-709q4or9
  Resolved https://github.com/stanfordnlp/dsp.git to commit 238e31211474cc6c3de917aed9eaccfe911132f0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [65]:
from openai import OpenAI
import numpy as np
def my_vectorizer(query):
    client = OpenAI(
      api_key=os.environ["NV_DEV_API_KEY"],
      base_url="https://integrate.api.nvidia.com/v1"
    )

    response = client.embeddings.create(
        input=query,
        model="nvdev/nvidia/nv-embed-v1",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )
    embedding=np.array(response.data[0].embedding)
    
    return embedding
embedding=my_vectorizer(example.question)


(numpy.ndarray, (4096,))

In [74]:
from dspy.teleprompt import KNNFewShot

knn_teleprompter = KNNFewShot(3, trainset, vectorizer=my_vectorizer)
compiled_knn = knn_teleprompter.compile(MultiChoices(trainset), trainset=trainset)

In [68]:
example = trainset[0]
pred = compiled_knn(question = example.question)
print("Question: ", example.question)
print("Expected answer: ", example.answer)
print("Prediction: ", pred.answer)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 580.69it/s]


Bootstrapped 1 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
Question:  When plants undergo photosynthesis, a reaction produces sugar, oxygen, and water. During respiration, stored energy from the products of photosynthesis is converted to usable energy. In what form is the energy stored prior to use in respiration? 
Expected answer:   chemical energy 
Prediction:  chemical energy


In [70]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_testset = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_knn` program with the `answer_exact_match` metric.
#metric = dspy.evaluate.answer_exact_match

    
evaluate_on_testset(compiled_knn, validate_answer)


  0%|                                                                                                                                                                                     | 0/30 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 581.01it/s]

Bootstrapped 1 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
inspect reasoning >>>  During photosynthesis, plants convert light energy into chemical energy, which is stored in the form of glucose. This chemical energy is then used during respiration to produce ATP, which is the usable energy for plants. False
stripped strange characters in pred if any : chemical energy
gold answer:  chemical energy 
True
Average Metric: 1.00 / 30 (3.3%):   3%|████▋                                                                                                                                      | 1/30 [00:01<00:44,  1.54s/it]



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 589.09it/s]


Bootstrapped 1 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
inspect reasoning >>>  Lichens are symbiotic organisms made up of fungi and green algae or cyanobacteria. In this symbiotic relationship, the green algae provide the fungi with food in the form of carbohydrates, which are produced through photosynthesis. The fungi, in turn, provide the green algae with protection and support. False
stripped strange characters in pred if any : food
gold answer:  food 
True
Average Metric: 2.00 / 30 (6.7%):   7%|█████████▎                                                                                                                                 | 2/30 [00:06<01:36,  3.46s/it]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 614.37it/s]


Bootstrapped 1 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
inspect reasoning >>>  Gas-efficient engines use less fuel to produce the same amount of energy, thus reducing the amount of emissions produced by the engine. The reduction in emissions leads to less air pollution in cities, which can have a significant impact on the air quality and overall environment. False
stripped strange characters in pred if any : air pollution
gold answer:  air pollution. 
False
Average Metric: 27.00 / 30 (90.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:48<00:00,  3.60s/it]

2025/01/09 12:51:34 INFO dspy.evaluate.evaluate: Average Metric: 27 / 30 (90.0%)


,question,choices,example_answer,reasoning,pred_answer,validate_answer
0,"When plants undergo photosynthesis, a reaction produces sugar, oxy...","[ chemical energy , kinetic energy , light energy , heat energy]",chemical energy,"During photosynthesis, plants convert light energy into chemical e...",chemical energy,✔️ [True]
1,Lichens are symbiotic organisms made of green algae and fungi. Wha...,"[ carbon dioxide , food , protection , water]",food,Lichens are symbiotic organisms made up of fungi and green algae o...,food,✔️ [True]
2,Which best describes why winds at the beach change direction betwe...,"[' because the temperatures over land and over water change ', ' b...",because the temperatures over land and over water change,"During the day, land heats up faster than water due to its lower h...",because the temperatures over land and over water change,✔️ [True]
3,"Many natural rock formations change color over time. In Utah, for ...","[ chemical weathering , mechanical weathering , water erosion , ...",chemical weathering,Chemical weathering involves the breakdown of rocks due to chemica...,chemical weathering,✔️ [True]
4,A cow and a frog have similar bone structures in their forelimbs. ...,"[' Both have undergone similar mutations. ', ' Both descended from...",Both descended from a common ancestor.,The similarity in bone structure between a cow and a frog is a res...,Both descended from a common ancestor.,✔️ [True]


90.0

---
## start optimizing using MIPROv2 

In [77]:
kwargs = dict(teacher_settings=dict(lm=mixtral8x22b), prompt_model=mixtral8x22b, max_errors=999)
# Import the optimizer
from dspy.teleprompt import MIPROv2

# Initialize optimizer
teleprompter = MIPROv2(
    metric=validate_answer,
    auto="light", # Can choose between light, medium, and heavy optimization runs
)

# Optimize program
print(f"Optimizing zero-shot program with MIPRO...")
optimized_program = teleprompter.compile(
    MultiChoices(trainset).deepcopy(),
    trainset=trainset,
    max_bootstrapped_demos=3, # ZERO FEW-SHOT EXAMPLES
    max_labeled_demos=1, # ZERO FEW-SHOT EXAMPLES
    requires_permission_to_run=False,
)

2025/01/09 12:57:38 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 24

2025/01/09 12:57:38 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/09 12:57:38 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/09 12:57:38 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Optimizing zero-shot program with MIPRO...
Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 17%|█████████████████████████████                                                                                                                                                 | 1/6 [00:01<00:09,  1.86s/it]

inspect reasoning >>>  Photosynthesis results in the production of sugar, oxygen, and water from carbon dioxide and water using light energy. This sugar is stored in the plant as chemical energy until it is needed during respiration, where it is converted to usable energy. False
stripped strange characters in pred if any : chemical energy
gold answer:  chemical energy 
True


 33%|██████████████████████████████████████████████████████████                                                                                                                    | 2/6 [00:02<00:05,  1.42s/it]

...

2025/01/09 12:58:40 INFO dspy.evaluate.evaluate: Average Metric: 22 / 24 (91.7%)
2025/01/09 12:58:40 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 91.67 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/01/09 12:58:40 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.5, 91.67, 83.33, 83.33, 83.33, 91.67]
2025/01/09 12:58:40 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 91.67
2025/01/09 12:58:40 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/09 12:58:40 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



  0%|                                                                                                                                                                                     | 0/24 [00:00<?, ?it/s]inspect reasoning >>>  Plant decomposition contributes significantly to the formation of dark, fertile soil by enriching it with organic matter and nutrients essential for plant growth. False
stripped strange characters in pred if any : plant decomposition
gold answer:  plant decomposition 
True
Average Metric: 1.00 / 1 (100.0%):   4%|█████▊                                                                                                                                    | 1/24 [00:01<00:27,  1.21s/it]inspect reasoning >>>  Rocks are classified into igneous, metamorphic, and sedimentary based on how they formed. Igneous rocks form from the cooling of magma or lava, metamorphic rocks form from existing rocks that undergo changes due to heat and pressure, and sedimentary rocks form 

2025/01/09 12:58:47 INFO dspy.evaluate.evaluate: Average Metric: 22 / 24 (91.7%)
2025/01/09 12:58:47 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 91.67 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/09 12:58:47 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.5, 91.67, 83.33, 83.33, 83.33, 91.67, 91.67]
2025/01/09 12:58:47 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 91.67
2025/01/09 12:58:47 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/09 12:58:47 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



  0%|                                                                                                                                                                                     | 0/24 [00:00<?, ?it/s]inspect reasoning >>>  New crust is formed at divergent plate boundaries, where two tectonic plates move away from each other. This movement allows magma to rise from the mantle and solidify into new crust, creating features such as mid-ocean ridges and rift valleys. False
stripped strange characters in pred if any : divergent
gold answer:  divergent
True
Average Metric: 1.00 / 1 (100.0%):   4%|█████▊                                                                                                                                    | 1/24 [00:02<00:48,  2.11s/it]inspect reasoning >>>  Rocks are classified into igneous, metamorphic, and sedimentary based on how they formed. Igneous rocks form from the cooling and solidification of molten magma or lava. Metamorphic rocks form through

2025/01/09 12:59:06 INFO dspy.evaluate.evaluate: Average Metric: 22 / 24 (91.7%)
2025/01/09 12:59:06 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 91.67 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/09 12:59:06 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.5, 91.67, 83.33, 83.33, 83.33, 91.67, 91.67, 91.67]
2025/01/09 12:59:06 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 91.67
2025/01/09 12:59:06 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/09 12:59:06 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 91.67!


In [78]:
## saving the optimized program
optimized_program.save(f"/workspace/nvdata/mipro_optimized.json")
